In [ ]:
!curl -L https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip -o data/PennFudanPed.zip
!cd data && unzip -o PennFudanPed.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51.2M  100 51.2M    0     0  2104k      0  0:00:24  0:00:24 --:--:-- 1639k


In [11]:
!cd data && unzip -o PennFudanPed.zip

Archive:  PennFudanPed.zip
  inflating: PennFudanPed/added-object-list.txt  
  inflating: PennFudanPed/Annotation/FudanPed00001.txt  
  inflating: PennFudanPed/Annotation/FudanPed00002.txt  
  inflating: PennFudanPed/Annotation/FudanPed00003.txt  
  inflating: PennFudanPed/Annotation/FudanPed00004.txt  
  inflating: PennFudanPed/Annotation/FudanPed00005.txt  
  inflating: PennFudanPed/Annotation/FudanPed00006.txt  
  inflating: PennFudanPed/Annotation/FudanPed00007.txt  
  inflating: PennFudanPed/Annotation/FudanPed00008.txt  
  inflating: PennFudanPed/Annotation/FudanPed00009.txt  
  inflating: PennFudanPed/Annotation/FudanPed00010.txt  
  inflating: PennFudanPed/Annotation/FudanPed00011.txt  
  inflating: PennFudanPed/Annotation/FudanPed00012.txt  
  inflating: PennFudanPed/Annotation/FudanPed00013.txt  
  inflating: PennFudanPed/Annotation/FudanPed00014.txt  
  inflating: PennFudanPed/Annotation/FudanPed00015.txt  
  inflating: PennFudanPed/Annotation/FudanPed00016.txt  
  inflating

In [12]:
import os
import numpy as np
from PIL import Image
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt

In [ ]:
#@title PennFudanPed Dataset
class PennFudanDataset(Dataset):
    def __init__(self, root, transform=None):
        '''
        root: pennfudanped 폴더의 상위 경로 (예: "./data/PennFudanPed")
        transform: 이미지 및 마스크에 적용할 transform (동일하게 젹용)
        '''
        self.root = root                                            # 데이어셋이 저장된 루트 디렉토리
        self.imgs_dir = os.path.join(root, "PNGImages")             # 이미지 파일이 들어 있는 폴더 연결
        self.masks_dir = os.path.join(root, "PedMasks")             # 마스크 파일이 들어 있는 폴더 연결
        self.imgs = list(sorted(os.listdir(self.imgs_dir)))         # 이미지 파일 목록 정렬
        self.masks = list(sorted(os.listdir(self.masks_dir)))       # 마스크 파일 목록 정렬
        self.transform = transform                                  # 변환 함수 (augmentation이나 resize등 적용 가능)
    
    def __len__(self):
        return len(self.imgs)                                       # 전체 데이터셋의 이밎 개수 반환
    
    def __getitem__(self, idx):
        #이미지로드
        img_path = os.path.join(self.imgs_dir, self.imgs[idx])      # idx번째 이미지 경로
        mask_path = os.path.join(self.masks_dir, self.masks[idx])   # idx번째 마스크 경로
        img = Image.open(img_path).convert("RGB")                   # 이미지를 RGB 모드로 열기
        mask = Image.open(mask_path)                                # 마스크 이미지는 흑백/팔레트 모드 그대로 열기

        # mask는 각 인스턴스마다 다른 값이지만, 이전 분할을 위해
        # 0은 배경, 그 외는 보행자(1)로 처리
        mask = np.array(mask)                                       # mask를 numpy 배열로 변환
        # 여러 인스턴스 픽셀값 >> 0을 1로 변경
        mask = np.where(mask > 0, 1, 0).astype('uint8')             # 0과 1로 이진화 (사람 = 1, 배경 = 0)
        mask = Image.fromarrary(mask)                               # 다시 PIL 이미지로 변환

        # transform 적용(동일하게 이미지와 마스크에 적용)
        if self.transform is not None:                              # 이미지와 마스크에 같은 transform 적용
            # mask의 경우 PIL.Image.NEAREST를 사용해야 함
            img, mask = self.transform(img, mask)

        return img, mask

In [ ]:
#@title Transform

# 예시: transform 함수 (resize, tensor 변환)
def joint_transform(img, mask, size=(256, 256)):                    #
    # Resize: 이미지와 mask에 동일하게 적용
    img = TF.resize(img, size)
    mask = TF.resize(mask, size, interpolation=Image.NEAREST)
    # ToTensor: 이미지는 [0,1] 실수 tensor, mask는 int tensor
    img = TF.to_tensor(img)
    mask = torch.as_tensor(np.array(mask), dtype=torch.long)
    return img, mask

In [15]:
#@title UNet 모델 정의

class DoubleConv(nn.Module):
    '''(convolution >> [BN] >> ReLU) * 2'''
    def init__(self, in_ch, out_ch):
        super().__init__()                                              # nn.Module 내부 초기화(파라미터 추적/이동 준비)
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernael_size=3, padding=1),        # 3x3 conv(크기유지)원래 UNet 에서는 패딩을 추가하지 않음
            nn.BatchNorm2d(out_ch),                                     # 배치 정규화(학습 안정화)
            nn.ReLU(True),                                              # 비선형성(in-place)

            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(True)
        )
    def forawrd(self, x):
        return self.double_conv(x)                                      # 두 번의 conv-BN-ReLU 통과 결과
    
class UNet(nn.Module):
    def __init__(self, n_ch=3, n_classes=2):
        super().__init__()
        # ======== 인코더 ========
        self.inc = DoubleConv(n_ch, 64)             # 입력 >> 64채널 (H, W 유지)

        self.down1 = nn.Sequential(
            nn.maxpool2d(2),                        # 해상도 1/2 (H/2, W/2)
            DoubleConv(64, 128)                     # 채널 64 >> 128
        )
        self.down2 = nn.Sequential(
            nn.MaxPool2d(2),                        # 해상도 1/4 (H/4, W/4)
            DoubleConv(128, 256)                    # 채널 128 >> 256
        )
        self.down3 = nn.Sequential(
            nn.MaxPool2d(2),                        # 해상도 1/8 (H/8, W/8)
            DoubleConv(256, 512)                    # 채널 256 >> 512
        )
        self.down4 = nn.Sequential(
            nn.MaxPool2d(2),                        # 해상도 1/16 (H/16, W/16)
            DoubleConv(512, 512)                    # 채널 512 >> 512 (깊은 특징)
        )
        # ======== 디코더 ========
        # 업샘플 : ConvTranspose2d(디컨브)로 해상도 2배
        self.up1 = nn.ConvTranspose2d(512, 512, kernel_size=2, stride=2)            #(H/8, W/8)
        self.conv1 = DoubleConv(1024, 256)          # x4와 concat >> 1024 >> 256 으로 압축

        self.up2 = nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2)            #(H/4, W/4)
        self.conv2 = DoubleConv(512, 128)          # x3와 concat >> 512 >> 128 으로 압축

        self.up3 = nn.ConvTranspose2d(128, 128, kernel_size=2, stride=2)            #(H/2, W/2)
        self.conv3 = DoubleConv(256, 64)           # x2와 concat >> 256 >> 64 으로 압축

        self.up4 = nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2)            #(H, W)
        self.conv4 = DoubleConv(128, 64)           # x4와 concat >> 128 >> 64 으로 압축

        # 출력층
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)                         # 1x1 conv >> 클래스 로짓

    def forward(self, x):
        # ======== 인코더 ========
        x1 = self.inc(x)                        # (B, 64, H, W)
        x2 = self.down1(x1)                     # (B, 128, H/2, W/2)
        x3 = self.down1(x2)                     # (B, 256, H/4, W/4)
        x4 = self.down1(x3)                     # (B, 512, H/8, W/8)
        x5 = self.down1(x4)                     # (B, 512, H/16, W/16)

        # ======== 디코더 ========
        x  = self.up1(x5)             # (B, 512, H/8, W/8)
        x  = torch.cat([x, x4], dim=1)# 채널 결합: 512+512=1024
        x  = self.conv1(x)            # (B, 256, H/8, W/8)

        x  = self.up2(x)              # (B, 256, H/4, W/4)
        x  = torch.cat([x, x3], dim=1)# 256+256=512
        x  = self.conv2(x)            # (B, 128, H/4, W/4)

        x  = self.up3(x)              # (B, 128, H/2, W/2)
        x  = torch.cat([x, x2], dim=1)# 128+128=256
        x  = self.conv3(x)            # (B, 64,  H/2, W/2)

        x  = self.up4(x)              # (B, 64,  H,   W)
        x  = torch.cat([x, x1], dim=1)# 64+64=128
        x  = self.conv4(x)            # (B, 64,  H,   W)

        logits = self.outc(x)         # (B, n_classes, H, W) — 소프트맥스/시그모이드는 밖에서!
        return logits
